In [6]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[2], 'GPU')
  except RuntimeError as e:
    # Visible devices must be set at program startup
    print(e)

In [7]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle
import cv2
import pandas as pd

In [ ]:
print("[INFO] loading autoencoder and image data...")
autoencoder = load_model('modelCitologia')
images = pickle.loads(open('trainDatasetImages', "rb").read())
images = images.astype("float32") / 255.0
# make predictions on our image data and initialize our list of
# reconstruction errors
decoded = autoencoder.predict(images)
errors = []
# loop over all original images and their corresponding
# reconstructions
for (image, recon) in zip(images, decoded):
	# compute the mean squared error between the ground-truth image
	# and the reconstructed image, then add it to our list of errors
	mse = np.mean((image - recon) ** 2)
	errors.append(mse)

In [13]:
thresh = np.quantile(errors, 0.999)

In [14]:
thresh

0.024599509194493167

In [5]:
print("[INFO] loading autoencoder and image data from the test set...")
images = pickle.loads(open('testDatasetImages', "rb").read())
images = images.astype("float32") / 255.0
# make predictions on our image data and initialize our list of
# reconstruction errors
decoded = autoencoder.predict(images)
testErrors = []
# loop over all original images and their corresponding
# reconstructions
for (image, recon) in zip(images, decoded):
	# compute the mean squared error between the ground-truth image
	# and the reconstructed image, then add it to our list of errors
	mse = np.mean((image - recon) ** 2)
	testErrors.append(mse)

[INFO] loading autoencoder and image data from the test set...


UnknownError: 2 root error(s) found.
  (0) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node encoder_1/conv2d_2/Conv2D}}]]
  (1) Unknown: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[{{node encoder_1/conv2d_2/Conv2D}}]]
	 [[decoder_1/activation_1/Sigmoid/_1107]]
0 successful operations.
0 derived errors ignored.

In [6]:
thresh = 0.05

In [15]:
idxs = np.where(np.array(testErrors) >= thresh)[0]

In [16]:
res = []
testNames = pickle.loads(open('testDatasetNames', "rb").read())
for idx in idxs:
    res.append([testNames[idx],testErrors[idx]])
df = pd.DataFrame(res,columns=['name','error'])

In [17]:
def extractName(string):
    return string[string[4:].find('/')+5:string.find(' ')]

In [18]:
df['imageName'] = df['name'].apply(lambda x: extractName(x))

In [19]:
df.groupby('imageName').size()

imageName
1005124.tif    275
1005125.tif    160
1005126.tif     41
1005127.tif     16
1005128.tif    662
1005129.tif    562
1005130.tif    183
1005131.tif     24
1005132.tif     30
1005133.tif    369
1005205.tif     31
1005206.tif     15
1005207.tif     86
1005209.tif     11
1005210.tif    125
1005211.tif     76
1005212.tif     17
13I.tif          2
14I.tif         47
20I.tif         60
29I.tif          1
39I.tif          3
75I.tif          1
dtype: int64

In [12]:
np.unique([extractName(x) for x in testNames])

array(['1005124.tif', '1005125.tif', '1005126.tif', '1005127.tif',
       '1005128.tif', '1005129.tif', '1005130.tif', '1005131.tif',
       '1005132.tif', '1005133.tif', '1005134.tif', '1005205.tif',
       '1005206.tif', '1005207.tif', '1005208.tif', '1005209.tif',
       '1005210.tif', '1005211.tif', '1005212.tif', '13I.tif', '14I.tif',
       '20I.tif', '23I.tif', '26I.tif', '29I.tif', '2I.tif', '35I.tif',
       '39I.tif', '46I.tif', '50I.tif', '55I.tif', '60I.tif', '69I.tif',
       '73I.tif', '75I.tif', '85I.tif', '86I.tif', '87I.tif'],
      dtype='<U11')

In [19]:
thresh = 0.08
idxs = np.where(np.array(testErrors) >= thresh)[0]

In [20]:
len(idxs)

17

In [22]:
outputs = None
for i in idxs:
	# grab the original image and reconstructed image
	original = (images[i] * 255).astype("uint8")
	recon = (decoded[i] * 255).astype("uint8")
	# stack the original and reconstructed image side-by-side
	output = np.hstack([original, recon])
	# if the outputs array is empty, initialize it as the current
	# side-by-side image display
	if outputs is None:
		outputs = output
	# otherwise, vertically stack the outputs
	else:
		outputs = np.vstack([outputs, output])
# show the output visualization
cv2.imwrite("Output.jpg", outputs)

True

In [35]:
thresh = 0.00008
idxs = np.where(np.array(testErrors) <= thresh)[0]

In [36]:
len(idxs)

7

In [37]:
outputs = None
for i in idxs:
	# grab the original image and reconstructed image
	original = (images[i] * 255).astype("uint8")
	recon = (decoded[i] * 255).astype("uint8")
	# stack the original and reconstructed image side-by-side
	output = np.hstack([original, recon])
	# if the outputs array is empty, initialize it as the current
	# side-by-side image display
	if outputs is None:
		outputs = output
	# otherwise, vertically stack the outputs
	else:
		outputs = np.vstack([outputs, output])
# show the output visualization
cv2.imwrite("OutputWellReconstructed.jpg", outputs)

True